In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from tkinter import *
from tkinter.messagebox import showinfo
from gtts import gTTS
import speech_recognition as sr
import os


# In[2]:


from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
from keras.models import load_model
from keras.preprocessing import image


# In[3]:


import cv2
import csv
import numpy as np
import pandas as pd
import datetime
import time
import pyttsx3


# In[4]:


from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
from PIL import Image, ImageTk



# In[ ]:


mainwindow= Tk()
mainwindow.title('Face recognition')
mainwindow.geometry('1200x1200')
mainwindow.resizable(0, 0)
p = PhotoImage(file=r"E:\projects\AUTOMATIC\Automatic Interview\wp10281364.png")
l = Label(image=p)
l.place(x=0,y=0)
path = r"E:\projects\AUTOMATIC\Automatic Interview\face-trainner.yml"

# In[ ]:


def face():
    global na
    na = "unknown"
    labels = ["pavan", "Soud","Waseem"] 

    face_cascade = cv2.CascadeClassifier(r'C:\Users\sujit\Downloads\haarcascade_frontalface_default.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(path)

    cap = cv2.VideoCapture(0) #Get vidoe feed from the Camera

    while(True):

        ret, img = cap.read() # Break video into frames 
        gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert Video frame to Greyscale
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5) #Recog. faces
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w] #Convert Face to greyscale 

            id_, conf = recognizer.predict(roi_gray) #recognize the Face

            if conf>=80:
                font = cv2.FONT_HERSHEY_SIMPLEX #Font style for the name 
                name = labels[id_] #Get the name from the List using ID number 
                na = name
                cv2.putText(img, name, (x,y), font, 1, (0,0,255), 2)

            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

        cv2.imshow('Preview',img) #Display the Video
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
     # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    #print(na)
    if na in labels:
        nex = Toplevel(mainwindow)
        nex.title('Embedded programs')
        nex.geometry("600x600")
        p = PhotoImage(file=r"C:\Users\sujit\Downloads\beautiful-wavy-background_23-2148465605.png")
        l = Label(nex,image=p)
        l.place(x=0,y=0)
        
        Label(nex, text="welcome " + na, font=("Times New Roman", 25), bg='white').place(x=200, y=50)
        texttospeechbutton = Button(nex, text='Text-To-Speech Conversion', font=('Times New Roman', 16), bg='Cyan', command=TextToSpeech)
        texttospeechbutton.place(x=50, y=250)

        speechtotextbutton = Button(nex, text='Speech-To-Text Conversion', font=('Times New Roman', 16), bg='Cyan', command=SpeechToText)
        speechtotextbutton.place(x=50, y=400)

        c = Button(nex,text="eye ball tracking",font=('Times New Roman', 16), bg='Cyan',command=eye).place(x=400, y=250)
        d = Button(nex,text="emotion detector",font=('Times New Roman', 16), bg='Cyan',command=emotion).place(x=400, y=400) 
        f = Button(nex,text="Quit",font=('Times New Roman', 15), bg='Red',command=nex.destroy).place(x=500, y=500)
        nex.mainloop()
        


   


# In[8]:


def eye():
    eye_cascade = cv2.CascadeClassifier('eye.xml')

    cap = cv2.VideoCapture(0)

    while 1:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        eyes = eye_cascade.detectMultiScale(gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            roi_gray2 = gray[ey:ey+eh, ex:ex+ew]
            roi_color2 = img[ey:ey+eh, ex:ex+ew]
            circles = cv2.HoughCircles(roi_gray2,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius=0,maxRadius=0)
            try:
                for i in circles[0,:]:
                    # draw the outer circle
                    #cv2.circle(roi_color2,(int(i[0]),int(i[1])),int(i[2]),(255,255,255),2)
                    cv2.circle(roi_color2,(i[0],i[1]),i[2],(255,255,255),2)
                    print("drawing circle")
                    # draw the center of the circle
                    #cv2.circle(roi_color2,(int(i[0]),int(i[1])),2,(255,255,255),3)
                    cv2.circle(roi_color2,(i[0],i[1]),2,(255,255,255),3)
            except Exception as e:
                pass
        cv2.imshow('img',img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# In[9]:


def emotion():
    face_classifier = cv2.CascadeClassifier(r'C:\Users\sujit\Downloads\haarcascade_frontalface_default.xml')
    classifier =load_model('Emotion_little_vgg.h5')

    class_labels = ['Angry','Happy','Neutral','Sad','Surprise']


    cap = cv2.VideoCapture(0)



    while True:
        # Grab a single frame of video
        ret, frame = cap.read()
        labels = []
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)

        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h,x:x+w]
            roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        # rect,face,image = face_detector(frame)


            if np.sum([roi_gray])!=0:
                roi = roi_gray.astype('float')/255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi,axis=0)

            # make a prediction on the ROI, then lookup the class

                preds = classifier.predict(roi)[0]
                label=class_labels[preds.argmax()]
                label_position = (x,y)
                cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
            else:
                cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        cv2.imshow('Emotion Detector',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# In[10]:


def say(text1):
        language = 'en'
        speech = gTTS(text = text1, lang = language, slow = False)
        speech.save("text.mp3")
        os.system("start text.mp3")

def recordvoice():
    while True:
        r = sr.Recognizer()
        with sr.Microphone() as source:
            audio=r.listen(source)
            try:    
                text1 = r.recognize_google(audio,language="en-IN")
            except:
                pass
            return text1


# In[ ]:


def TextToSpeech():
    texttospeechwindow = Toplevel(mainwindow)
    texttospeechwindow.title('Text-to-Speech')
    texttospeechwindow.geometry("500x500")
    texttospeechwindow.configure(bg='Blue')
 
    Label(texttospeechwindow, text='Text-to-Speech Converter', font=("Times New Roman", 15), bg='Blue').place(x=50)
 
    text = Text(texttospeechwindow, height=5, width=30, font=12)
    text.place(x=7, y=60)
   
    speakbutton = Button(texttospeechwindow, text='listen', bg='coral', command=lambda: say(str(text.get(1.0, END))))
    speakbutton.place(x=140, y=200)

def SpeechToText():
    speechtotextwindow = Toplevel(mainwindow)
    speechtotextwindow.title('Speech-to-Text Converter by')
    speechtotextwindow.geometry("500x500")
    speechtotextwindow.configure(bg='pink')
    Label(speechtotextwindow, text='Speech-to-Text', font=("Comic Sans MS", 15), bg='IndianRed').place(x=50)
 
    text = Text(speechtotextwindow, font=12, height=3, width=30)
    text.place(x=7, y=100)
   
    recordbutton = Button(speechtotextwindow, text='Record', bg='Sienna', command=lambda: text.insert(END, recordvoice()))
    recordbutton.place(x=140, y=50)

Label(mainwindow, text='Wanna recognize your face!!',
     font=('Times New Roman', 25), bg='white', wrap=True, wraplength=450).place(x=400, y=50)

e = Button(mainwindow,text="face recognition ",font=('Times New Roman', 20), bg='White',command=face).place(x=500, y=350)
g = Button(mainwindow,text="Quit",font=('Times New Roman', 15), bg='Red',command=mainwindow.destroy).place(x=1100, y=700)

mainwindow.update()
mainwindow.mainloop()


# In[ ]:






TclError: encountered an unsupported criticial chunk type "orNT"